### В данном уроке мы выделим еще несколько потенциально важных фичей из изначального датасета и попробуем применить изученные методы отбора признаков к итоговому датасету.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

processed_data = pd.read_csv('processed_data.csv', index_col='id')

processed_data.head()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


In [2]:
initial_data = pd.read_csv('taxi_dataset.csv')

initial_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N


In [3]:
initial_data.shape[0] == processed_data.shape[0]

True

In [4]:
### Вернем в датасет колонку pickup_datetime

initial_data = initial_data.set_index('id')

processed_data = pd.merge(processed_data, initial_data['pickup_datetime'],
                          left_index=True, right_index=True)

In [5]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime
id,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55


Напомним, **pickup_datetime** - время начала поездки.

Кажется, что в зависимости от месяца/дня недели/времени суток движение на дорогах может отличаться. Как из-за погодных условий, так и из-за загруженности транспорта. Поэтому, есть подозрение, что будет полезно выделить ряд признаков из колонки **pickup_datetime**. Давайте исследуем зависимость нашей таргетной переменной от указанных факторов.

In [6]:
processed_data['pickup_datetime'] = pd.to_datetime(processed_data['pickup_datetime'])

processed_data['date'] = processed_data.pickup_datetime.dt.date
processed_data['day_of_week'] = processed_data.pickup_datetime.dt.dayofweek
processed_data['hour'] = processed_data.pickup_datetime.dt.hour
processed_data['month'] = processed_data.pickup_datetime.dt.month

In [7]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime,date,day_of_week,hour,month
id,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55,2016-03-14,0,17,3
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35,2016-06-12,6,0,6
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24,2016-01-19,1,11,1
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31,2016-04-06,2,19,4
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55,2016-03-26,5,13,3


Исследуем, когда и сколько поездок было совершено. 

Начнем с графиков, показывающих количество поездок в зависимости от времени суток/даты и т.д.

Так же полезно сразу показать и среднее значение таргетной переменной.

In [8]:
# trip_by_month = processed_data.groupby(['month', 'day_of_week'])['log_trip_duration'].mean().reset_index().groupby('month')['log_trip_duration'].mean()

In [9]:
# import matplotlib.pyplot as plt

# fig = plt.figure()

# fig.set_size_inches(16, 10)

# ax_1 = fig.add_subplot(1, 2, 1)
# plt.hist(processed_data['date'], bins=processed_data.date.unique().shape[0])
# plt.ylim((4000, 10000))

# ax_2 = fig.add_subplot(1, 2, 2)
# plt.bar(sorted(list(processed_data['date'].unique())), 
#         processed_data.groupby('date', as_index=False)['log_trip_duration'].mean()['log_trip_duration'],
#         width=1)
# plt.ylim((6, 7))

# fig.tight_layout()

# ax_1.set(title = 'Количество поездок по датам')
# ax_2.set(title = 'Средняя логарифмированная продолжительность поездок по датам')

# plt.show()

Кажется, что внутри недели есть некоторая зависимость таргета от дня недели. 

Такая же зависимость может оказаться и внутри дня (в завимости от часов).

Предлагаю исследовать эту зависимость подробнее.

Изобразите следующие графики зависимостей:

- Количество поездок/Средняя продолжительность поездки vs День недели
- Количество поездок/Средняя продолжительность поездки vs время суток
- Средняя продолжительность поездки vs время суток для каждого дня недели (например, нарисовав с разными цветами и, соответственно, легендами)
- Аналогичные графики, используя информацию о месяце, внутри которого была совершена поездка.
- Ящики с усами для различных: времени суток, дня недели, месяца

Используйте любые доступные инструменты pyplot!

Далее, на основе полученных результатов, мы будем принимать решение о создании новых признаков.

EDA можно использовать не только для того, чтобы понять, какие фичи можно убрать из датасета. Но и для выделения базовых признаков. Этим и займемся!

P.S. Сами графики мы проверять у Вас не будем. Зато для ответа на устные вопросы понадобятся, поэтому, в любом случае, крайне рекомендуем поупражняться. Поэтому рисовать все графики не обязательно - ограничьтесь теми, которые помогут Вам ответить на тестовые вопросы.

**Hint**: обратите внимание на сильную просадку в январе. Почему она могла произойти - можно прочитать <a href="https://en.wikipedia.org/wiki/January_2016_United_States_blizzard"> здесь</a>. В будущем можно будет создать бинарный признак "произошла ли поездка во время сильного снегопада".


In [10]:
# ### Your code is here
# fig = plt.figure()

# fig.set_size_inches(12, 7)

# ax_1 = fig.add_subplot(1, 2, 1)
# ax_1.plot(sorted(list(processed_data.day_of_week.unique())), processed_data.groupby('day_of_week').agg('count'), linestyle='-')
# # plt.ylim((4000, 10000))

# ax_2 = fig.add_subplot(1, 2, 2)
# ax_2.plot(sorted(list(processed_data['day_of_week'].unique())), 
#         processed_data.groupby('day_of_week', as_index=False)['log_trip_duration'].mean()['log_trip_duration'])
# plt.ylim((6, 7))

# fig.tight_layout()

# ax_1.set(title = 'Количество поездок по дням недели')
# ax_2.set(title = 'Средняя логарифмированная продолжительность поездок по дням недели')

# plt.show()



In [11]:
# fig = plt.figure()

# fig.set_size_inches(12, 7)

# ax_1 = fig.add_subplot(1, 2, 1)
# ax_1.plot(sorted(list(processed_data.hour.unique())), processed_data.groupby('hour').agg('count'), marker='o', linestyle='-')
# # plt.ylim((4000, 10000))

# ax_2 = fig.add_subplot(1, 2, 2)
# ax_2.plot(sorted(list(processed_data['hour'].unique())), 
#         processed_data.groupby('hour', as_index=False)['log_trip_duration'].mean()['log_trip_duration'], marker='o', linestyle='--')
# plt.ylim((6, 7))

# fig.tight_layout()

# ax_1.set(title = 'Количество поездок по часам')
# ax_2.set(title = 'Средняя логарифмированная продолжительность поездок по часам')

# plt.show()

In [12]:
# trip_by_month = processed_data.groupby(['month', 'day_of_week'])['log_trip_duration'].mean().reset_index().groupby('month')['log_trip_duration'].mean()
# count_by_month = processed_data.groupby('month')['log_trip_duration'].count()

# fig = plt.figure()

# fig.set_size_inches(12, 7)

# ax_1 = fig.add_subplot(1, 2, 1)
# plt.bar(sorted(list(processed_data['month'].unique())), count_by_month)
# # plt.ylim((4000, 10000))

# ax_2 = fig.add_subplot(1, 2, 2)
# plt.bar(sorted(list(processed_data['month'].unique())), 
#         trip_by_month,
#         width=0.5)
# plt.ylim((6, 7))

# fig.tight_layout()

# ax_1.set(title = 'Количество поездок по месяцам')
# ax_2.set(title = 'Средняя логарифмированная продолжительность поездок по месяцам')

# plt.show()

Кажется, что полученные выводы из графиков выше достаточны для того, чтобы посчитать признаки, связанные с временем/датой поездки, достаточно важными. На их основе выделим еще пару.

Добавьте следующие бинарные признаки:


1. Была ли в день поездки буря (основываясь на единственном обнаруженном таком дне)
2. Является ли время поездки статистически самым проблемным, то есть час пиком?

Для создания 2ой фичи используйте следующую логику: посчитаем для каждой пары "день недели"-"время суток" среднее значение таргета. Найдем топ-10 самых "больших" пар. Если поездка была совершена во входящее в этот топ время, то ставим 1. Иначе - 0. Получается бинарный признак.

P.S. назовите колонки **anomaly** и **traffic_jam**

In [13]:
### Создадим первый бинарный признак
### Your code is here
avg_date = processed_data.groupby('date', as_index=False)['log_trip_duration'].count()
storm_date = avg_date[avg_date['log_trip_duration'] < 3000]['date'].item()
processed_data['anomaly'] = 0
processed_data.loc[processed_data['date'] == storm_date, 'anomaly'] = 1
processed_data['anomaly'].value_counts()

0    1456996
1       1648
Name: anomaly, dtype: int64

In [14]:
processed_data

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime,date,day_of_week,hour,month,anomaly
id,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55,2016-03-14,0,17,3,0
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35,2016-06-12,6,0,6,0
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24,2016-01-19,1,11,1,0
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31,2016-04-06,2,19,4,0
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55,2016-03-26,5,13,3,0
...,...,...,...,...,...,...,...,...,...,...,...
id2376096,1,1053.529749,0,1.226296,6.658011,2016-04-08 13:31:04,2016-04-08,4,13,4,0
id1049543,0,930.399753,0,6.056935,6.486161,2016-01-10 07:35:15,2016-01-10,6,7,1,0
id2304944,1,930.399753,0,7.832952,6.639876,2016-04-22 06:57:41,2016-04-22,4,6,4,0


In [15]:
top10 = processed_data.pivot_table(index='hour', columns='day_of_week', values='log_trip_duration').stack().nlargest(10).reset_index().sort_values(by=['day_of_week', 'hour'])

In [16]:
top10[['day_of_week', 'hour']]

,day_of_week,hour
6,1,11
3,2,11
4,2,13
5,2,14
2,2,15
7,3,11
0,3,14
1,3,15
9,3,16
8,4,15


In [17]:
(processed_data.groupby(['day_of_week', 'hour'])['log_trip_duration']
    .mean().reset_index().sort_values(by='log_trip_duration', ascending=False).head(10)
    .drop('log_trip_duration', axis=1)
    .reset_index(drop=True))

,day_of_week,hour
0,3,14
1,3,15
2,2,15
3,2,11
4,2,13
5,2,14
6,1,11
7,3,11
8,4,15
9,3,16


In [18]:
### Создадим второй бинарный признак
### Your code is here

rush = (processed_data.groupby(['day_of_week', 'hour'])['log_trip_duration']
    .mean().reset_index().sort_values(by='log_trip_duration', ascending=False).head(10)
    .drop('log_trip_duration', axis=1)
    .reset_index(drop=True)
    .sort_values(by=['day_of_week', 'hour']))

rush['traffic_jam'] = 1
rush

,day_of_week,hour,traffic_jam
6,1,11,1
3,2,11,1
4,2,13,1
5,2,14,1
2,2,15,1
7,3,11,1
0,3,14,1
1,3,15,1
9,3,16,1
8,4,15,1


In [19]:
rush = rush.set_index(['day_of_week', 'hour'])
rush

traffic_jam
day_of_week hour             
1           11              1
2           11              1
            13              1
            14              1
            15              1
3           11              1
            14              1
            15              1
            16              1
4           15              1

In [20]:
processed_data.shape

(1458644, 11)

In [21]:
processed_data.merge(rush, left_on=['day_of_week', 'hour'], right_index=True, how='left')

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime,date,day_of_week,hour,month,anomaly,traffic_jam
id,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55,2016-03-14,0,17,3,0,NaN
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35,2016-06-12,6,0,6,0,NaN
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24,2016-01-19,1,11,1,0,1.0
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31,2016-04-06,2,19,4,0,NaN
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55,2016-03-26,5,13,3,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
id2376096,1,1053.529749,0,1.226296,6.658011,2016-04-08 13:31:04,2016-04-08,4,13,4,0,NaN
id1049543,0,930.399753,0,6.056935,6.486161,2016-01-10 07:35:15,2016-01-10,6,7,1,0,NaN
id2304944,1,930.399753,0,7.832952,6.639876,2016-04-22 06:57:41,2016-04-22,4,6,4,0,NaN


In [22]:
processed_data = processed_data.merge(rush, left_on=['day_of_week', 'hour'], right_index=True, how='left')
processed_data['traffic_jam'] = processed_data['traffic_jam'].fillna(0)
processed_data['traffic_jam'] = processed_data['traffic_jam'].apply(int)
processed_data

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,pickup_datetime,date,day_of_week,hour,month,anomaly,traffic_jam
id,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,2016-03-14 17:24:55,2016-03-14,0,17,3,0,0
id2377394,0,930.399753,0,1.807119,6.498282,2016-06-12 00:43:35,2016-06-12,6,0,6,0,0
id3858529,1,930.399753,0,6.392080,7.661527,2016-01-19 11:35:24,2016-01-19,1,11,1,0,1
id3504673,1,930.399753,0,1.487155,6.063785,2016-04-06 19:32:31,2016-04-06,2,19,4,0,0
id2181028,1,930.399753,0,1.189925,6.077642,2016-03-26 13:30:55,2016-03-26,5,13,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
id2376096,1,1053.529749,0,1.226296,6.658011,2016-04-08 13:31:04,2016-04-08,4,13,4,0,0
id1049543,0,930.399753,0,6.056935,6.486161,2016-01-10 07:35:15,2016-01-10,6,7,1,0,0
id2304944,1,930.399753,0,7.832952,6.639876,2016-04-22 06:57:41,2016-04-22,4,6,4,0,0


Теперь колонки **pickup_datetime**, **date** можно убрать. А про оставшиеся **day_of_week**, **hour**, **month** необходимо подумать:

- С одной стороны, первые две можно убрать, так как на их основе была создана колонка **traffic_jam**
- С другой стороны, зависимость с колонкой **traffic_jam** нелинейная, поэтому можно попробовать использовать все фичи в комбинации. Конечно, лучше попробовать оба варианта и проэкспериментировать, но ради упрощения - оставим все колонки. Хоть **day_of_week**, **hour** и описываются числами, мы понимаем, что это скорее категориальные фичи. Потому что, например, отношение между 23:00 и 00:00 не такое же, как между числами 23 и 0. Закодируем их с помощью OneHotEncoder. 
- Последняя (**month**) очевидно категориальная. Можно закодировать ее тоже через OneHotEncoder.

In [23]:
processed_data = processed_data.drop(['pickup_datetime', 'date'], axis=1)

In [24]:
processed_data

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,day_of_week,hour,month,anomaly,traffic_jam
id,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,0,17,3,0,0
id2377394,0,930.399753,0,1.807119,6.498282,6,0,6,0,0
id3858529,1,930.399753,0,6.392080,7.661527,1,11,1,0,1
id3504673,1,930.399753,0,1.487155,6.063785,2,19,4,0,0
id2181028,1,930.399753,0,1.189925,6.077642,5,13,3,0,0
...,...,...,...,...,...,...,...,...,...,...
id2376096,1,1053.529749,0,1.226296,6.658011,4,13,4,0,0
id1049543,0,930.399753,0,6.056935,6.486161,6,7,1,0,0
id2304944,1,930.399753,0,7.832952,6.639876,4,6,4,0,0


In [25]:
encode_cols = ['day_of_week', 'hour', 'month']
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded = encoder.fit_transform(processed_data[encode_cols])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())


In [29]:
# processed_data = processed_data.reset_index()
pd.concat([processed_data, encoded_df], axis=1).set_index('id').drop(encode_cols, axis=1)

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,anomaly,traffic_jam,day_of_week_1,day_of_week_2,day_of_week_3,...,hour_19,hour_20,hour_21,hour_22,hour_23,month_2,month_3,month_4,month_5,month_6
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
id3858529,1,930.399753,0,6.392080,7.661527,0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id3504673,1,930.399753,0,1.487155,6.063785,0,0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
id2181028,1,930.399753,0,1.189925,6.077642,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id2376096,1,1053.529749,0,1.226296,6.658011,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
id1049543,0,930.399753,0,6.056935,6.486161,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id2304944,1,930.399753,0,7.832952,6.639876,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [30]:
processed_data = pd.concat([processed_data, encoded_df], axis=1).set_index('id')
processed_data = processed_data.drop(encode_cols, axis=1)


In [31]:
cols = processed_data.columns[7:]
processed_data[cols] = processed_data[cols].applymap(int)

In [32]:
processed_data.head().to_csv('step4_df.csv')

In [34]:
processed_data.dtypes

vendor_id               int64
passenger_count       float64
store_and_fwd_flag      int64
distance_km           float64
log_trip_duration     float64
anomaly                 int64
traffic_jam             int64
day_of_week_1           int64
day_of_week_2           int64
day_of_week_3           int64
day_of_week_4           int64
day_of_week_5           int64
day_of_week_6           int64
hour_1                  int64
hour_2                  int64
hour_3                  int64
hour_4                  int64
hour_5                  int64
hour_6                  int64
hour_7                  int64
hour_8                  int64
hour_9                  int64
hour_10                 int64
hour_11                 int64
hour_12                 int64
hour_13                 int64
hour_14                 int64
hour_15                 int64
hour_16                 int64
hour_17                 int64
hour_18                 int64
hour_19                 int64
hour_20                 int64
hour_21   

In [ ]:
### Делаем OneHotEncoding и конкатим с processed_data


for col in ['day_of_week', 'hour', 'month']:
    
    ### Your code is here

Что же, мы с Вами научились с помощью EDA визуализаций понимать важность признаков не только постфактум, но и до того, как сформировался финальный датасет - на этапе **выделения базовых фичей**.

Обратимся теперь к методам фильтрации - например, применим корреляционный анализ для одной пары фичей, чтобы понять, нет ли относительно этих колонок в нашем датасете избытка информации. 

Вспомним так же изначальные вещественные признаки - distance_km, passenger_count

Посчитайте корреляцию между ними. Есть ли какие-то основания для беспокойства?

In [36]:
### Your code is here

round(processed_data['distance_km'].corr(processed_data['passenger_count']), 3)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


0.017

In [ ]:
processed_data.head()

In [38]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

In [39]:
### На данный момент у нас 41 фича
### Представим, что хочется сократить их количество до 5.
### Воспользуемся для этим каким-нибудь методом обертки
### Например, метод прямого отбора

### Your code is here
X = processed_data.drop('log_trip_duration', axis=1)
y = processed_data['log_trip_duration']

model = LinearRegression()
selector = SequentialFeatureSelector(model, n_features_to_select=5)
selector.fit(X, y)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=5)

In [40]:
### Перечислите фичи, которые окажутся наиболее желанными
### Your code is here
selector.get_feature_names_out()

array(['distance_km', 'traffic_jam', 'day_of_week_6', 'hour_5', 'hour_6'],
      dtype=object)

In [42]:
### Представим, что мы с Вами ничего не знаем про Кросс-Валидацию и отложенную выборку
### Замерьте качество линейной регрессии на данных фичах
### Сильно ли оно отличается от полученного на Кросс-Валидации в прошлом уроке в ДЗ?

### Your code is here
model = LinearRegression()
model.fit(X[selector.get_feature_names_out()], y)
y_pred = model.predict(X[selector.get_feature_names_out()])
round(np.mean((y - y_pred)**2), 3)

0.415